In [ ]:
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
from IPython.display import display

In [ ]:
views = {
    "Raw Data": lambda x, y: x,
    "Percent Change": lambda x, y: x.pct_change(),
    "Rolling Average": lambda x, y: x.rolling(y).mean(),
    "Rolling Variance": lambda x, y: x.rolling(y).var(),
    "Rolling Standard Deviation": lambda x, y: x.rolling(y).var() ** 0.5,
    "Rolling Coefficient of Variation": lambda x, y: (x.rolling(y).var() ** 0.5)
    / (x.rolling(y).mean()),
}


def format_plotly(fig, calc, data, start, end):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    if calc:
        if len(calc) == 1:
            fig_title = f"{calc[0]} of {data} from {start.strftime('%Y/%m/%d')} to {end.strftime('%Y/%m/%d')}"
        else:
            fig_title = f"{', '.join(calc)} of {data} from {start.strftime('%Y/%m/%d')} to {end.strftime('%Y/%m/%d')}"
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        title={
            "text": fig_title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )


last_tickers = ""

In [ ]:
def view_chart(calculation, data, rolling, start, end, tickers):
    global last_tickers, df
    if tickers and tickers[-1] == ",":
        if tickers != last_tickers:
            df = yf.download(tickers, period="max", progress=False)
            last_tickers = tickers

        start_n = datetime(start.year, start.month, start.day)
        end_n = datetime(end.year, end.month, end.day)
        fig = go.Figure()
        for item in calculation:
            calcs = views[item](df[data], rolling)
            result = calcs.loc[(calcs.index >= start_n) & (calcs.index <= end_n)]
        
            if isinstance(result, pd.Series):
                plot = go.Scatter(x=result.index, y=result, mode="lines", name=f"{tickers.split(',')[0]} {item}")
                fig.add_trace(plot)
            else:
                for val in result:
                    plot = go.Scatter(x=result.index,y=result[val],mode="lines",name=f"{val.upper()} {item}")
                    fig.add_trace(plot)

        format_plotly(fig, calculation, data, start, end)
        # Jupyter-lab and Google Colab: do not use renderer argument
        # VS Code: ADD: renderer="cocalc"
        # Voila: ADD: renderer="notebook"
        if os.environ.get('SERVER_SOFTWARE','jupyter').startswith('voila'):
            fig.show(config={"showTips": False}, renderer="notebook")
        else:
            fig.show(config={"showTips": False})

w_auto = widgets.Layout(width="auto")
calc_widget = widgets.SelectMultiple(options=list(views.keys()), value=["Raw Data"], layout=w_auto)
data_widget = widgets.Dropdown(
    options=["Open", "Close", "High", "Low", "Volume"], value="Close", layout=w_auto
)
rolling_widget = widgets.Dropdown(options=list(range(2, 101)), value=60, layout=w_auto)
start_widget = widgets.DatePicker(value=datetime.today() - timedelta(days=365), layout=w_auto)
end_widget = widgets.DatePicker(value=datetime.today(), layout=w_auto)
tickers_widget = widgets.Text(value="TSLA,", layout=w_auto)
data_box = widgets.VBox([tickers_widget, data_widget, rolling_widget])
date_box = widgets.VBox([start_widget, end_widget])
controls = widgets.HBox([calc_widget, data_box, date_box], layout=widgets.Layout(width="60%"))

stocks_view = widgets.interactive_output(
    view_chart,
    {"calculation": calc_widget,
    "data": data_widget,
    "rolling": rolling_widget,
    "start": start_widget,
    "end": end_widget,
    "tickers": tickers_widget,
    }
)

title_html = '<h1>Stock Analysis Dashboard</h1>'
warning_html = '<p style="color:red"=>You MUST use a comma after EVERY stock you type in.</p>'
app_contents = [widgets.HTML(title_html), controls, widgets.HTML(warning_html), stocks_view]
app = widgets.VBox(app_contents)


display(app)
